In [1]:
import requests
import pandas as pd

def get_badges(username, offset=0):
    url = f"https://app2.vagalume.com.br/ajax/playlisteiros_score.php?action=userBadge&login={username}&offset={offset}"
    with requests.Session() as s:
        r = s.get(url, headers={'Connection': 'close'})
        badges = r.json()

    return badges

## Dados do usuário

Preencher com o seu usuário e a quantidade de selos que possui atualmente.

In [2]:
user = 'wenneton'
quantidade_selos = 1300
paginas = int(quantidade_selos/300) + 1

## Busca por selos

Ao executar a célula abaixo o código irá procurar pelos selos que você possui no servidor do vaga. Essa parte leva em torno de 2 minutos pra executar.

In [3]:
offsets = [i*300 for i in range(paginas)]
selos_por_artista = {}
selos_repetidos = {}
selos_total = {}
count = 0
for offset in offsets:
    badges = get_badges(user, offset)
    for playlist in badges:
        selos = playlist['vincs']
        for selo in selos:
            artist = selo['band_descr']
            song = selo['pointer_name']
            count += 1
            if artist not in selos_por_artista.keys():
                selos_por_artista[artist] = [song]
            else:
                if song not in selos_por_artista[artist]:
                    selos_por_artista[artist].append(song)
                    selos_por_artista[artist].sort()
                else:
                    if artist not in selos_repetidos:
                        selos_repetidos[artist] = [song]
                    else:
                        selos_repetidos[artist].append(song)
                        selos_repetidos[artist].sort()
            if artist not in selos_total.keys():
                selos_total[artist] = [song]
            else:
                selos_total[artist].append(song)
                selos_total[artist].sort()

## Verificar selos

Depois de rodar as células de cima pelo menos uma vez, basta alterar o nome do artista que deseja verificar abaixo para uma nova checagem! :D

### Selos únicos por artista

In [22]:
artista = 'Finneas'

if artista in selos_por_artista.keys():
    print('Selos únicos com o(a) artista: ')
    print(selos_por_artista[artista])
    print('Quantidade de selos: ', len(selos_por_artista[artista]))
else:
    print('Você não tem selo com esse artista ou a grafia está errada!')

Selos únicos com o(a) artista: 
['Break My Heart Again', "Let's Fall in Love for the Night", 'Partners in Crime', 'Shelter', "What They'll Say About Us"]
Quantidade de selos:  5


### Selos repetidos

In [16]:
artista = 'Taylor Swift'

if artista in selos_repetidos.keys():
    print('Selos repetidos com o(a) artista: ')
    print(selos_repetidos[artista])
else:
    print('Você não tem selos repetidos com esse artista ou a grafia está errada!')

Selos repetidos com o(a) artista: 
['All Too Well']


## Mostrar todos os selos

Gera um arquivo chamado ```selos.csv``` na pasta com todos os selos que você possui em ordem alfabética.

In [42]:
artistas = list(selos_total.keys())
artistas.sort()
artistas_list = []
songs_list = []

for artista in artistas:
    for song in selos_total[artista]:
        artistas_list.append(artista)
        songs_list.append(song)

df = pd.DataFrame({'artista': artistas_list, 'musica': songs_list}, columns=['artista', 'musica'])
df.to_csv('selos.csv', index=False, sep=';')

## Selos por artista

In [47]:
artistas = list(selos_total.keys())
count_badges = {}

for artista in artistas:
    count = len(selos_total[artista])
    if count not in count_badges.keys():
        count_badges[count] = [artista]
    else:
        count_badges[count].append(artista)

In [48]:
counts = list(count_badges.keys())
counts.sort(reverse=True)
artistas_list = []
counts_list = []

for count in counts:
    for artista in count_badges[count]:
        artistas_list.append(artista)
        counts_list.append(count)

df = pd.DataFrame({'artista': artistas_list, 'quantidade': counts_list}, columns=['artista', 'quantidade'])
df.to_csv('count_selos_por_artista.csv', index=False, sep=';')